In [25]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt #for visualization
import re
import time
from lxml import html
from lxml.etree import tostring
import pprint
from lxml.cssselect import CSSSelector
from lxml import html
import json
import xml.etree.ElementTree as ET  

Scrape trending YouTube videos (https://www.youtube.com/feed/trending) to get the following data on each movie:
•	URL,
•	title,
•	duration,
•	username,
•	views.
1.	Get the data using Selenium


In [26]:
url="https://www.youtube.com/feed/trending"

In [27]:
def youtube(url):
    browser = webdriver.Chrome()
    browser.get(url)
    browser.implicitly_wait(20)
    titles=[i.text for i in browser.find_elements_by_id("video-title")]
    links=[i.get_attribute('href') for i in browser.find_elements_by_id("video-title")]
    attributes=[i.get_attribute('aria-label') for i in browser.find_elements_by_id("video-title")]
    views=["".join(re.findall("\\s([0-9,]+)\\s+views",i)) for i in attributes]
    page = BeautifulSoup(browser.page_source,"lxml")
    durations_tex=[i.get_text()for i in page.find_all('span',class_="style-scope ytd-thumbnail-overlay-time-status-renderer")]
    durations=[re.sub("[^0-9:]","",i)for i in durations_tex]
    usernames=[i.text for i in browser.find_elements_by_xpath("//yt-formatted-string[@id='byline']/a")]
    browser.implicitly_wait(30)
    info=pd.DataFrame({"titles":titles,"links":links,"usernames":usernames,"durations":durations,
         "views":views})
    browser.close()
    return info



In [28]:
data= youtube(url)

In [30]:
def make_num(strings):
    """this fuction we will use to convert column view to numeris in order
    to calculate average"""
    strings=int(re.sub('[^0-9,]', "", strings).replace(",",""))
    return strings
def col_range(column):
    """we will use this to calculate calculate range"""
    maxi=column.max()
    mini=column.min()
    result=maxi-mini
    return result

In [31]:
data["views"]=data["views"].apply(make_num)## we need this to make numeric

In [32]:
type(data["views"][0])

numpy.int64

In [33]:
c_range=col_range(data["views"])
print("The average of views for trending videos is %d"%(data['views'].mean()))
print("The median of views for trending videos is %d"%(data['views'].median()))
print("The range of views for trending videos is %d"%(c_range))

The average of views for trending videos is 3001732
The median of views for trending videos is 1479229
The range of views for trending videos is 20400506


In [15]:
#saving it into csv
data.to_csv("Youtube.csv",index=False)


In [14]:
#saving it into json
with open("youtubeJs.json", "w+") as output_file:
    output_file.write(data.to_json())

In [10]:
def func(row):
    xml = ['<item>']
    for field in row.index:
        xml.append('  <field name="{0}">{1}</field>'.format(field, row[field]))
    xml.append('</item>')
    return '\n'.join(xml)

In [13]:
xml_file=data.apply(func, axis=1)

In [54]:
tree=html.document_fromstring(xml_file)

lxml.html.HtmlElement

In [19]:
def to_xml(df, filename=None, mode='w'):
    def row_to_xml(row):
        xml = ['<item>']
        for i, col_name in enumerate(row.index):
            xml.append('  <field name="{0}">{1}</field>'.format(col_name, row.iloc[i]))
        xml.append('</item>')
        return '\n'.join(xml)
    res = '\n'.join(df.apply(row_to_xml, axis=1))

    if filename is None:
        return res
    with open(filename, mode) as f:
        f.write(res)

In [23]:
to_xml(data,"file1.xml")##some error happened but I could not fixed it